# TODO:
1. Попробовать обучить разные модели для обычной задачи классификации
2. Поискать методы борьбы с дизбаллансом классов
3. Необходимо попробовать поставить задачу классификации как задачу обучения метрики
4. Задача обучения метрики с памятью
5. У нас есть доступ к тестовому множеству. Надо использовать это.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["PROJECT_DIR"]="/Max/Proga/VKMADE/homeworks/VK_CV/"

os.environ["N_ANCHOR_CLASSES"] = "10"

os.environ["IMG_H"] = "224"
os.environ["IMG_W"] = "224"

os.environ["MARGIN"] = "1.0"
os.environ["SESSION_SIZE"] = "100"

os.environ["MODEL_NAME"] = "smart_eff_twinlayer_derc"

In [3]:
import torch
import albumentations as albu
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from src.pipelines.train_model import train
from src.data.dataset import get_default_dataset
from src.architectures.swin import SwinNetModel
from src.architectures.effnet import EfficientNetModelB3
from src.data.save_load_model import save_model, load_model

from src.pipelines.predict import predict

In [4]:
dataset = get_default_dataset()

Reading images:   0%|          | 0/45373 [00:00<?, ?it/s]

In [5]:
EMBEDDING_SIZE = 600
model = EfficientNetModelB3(EMBEDDING_SIZE)
model.set_model_head_requires_grad()
model.set_model_to_finetune()
len(model.get_trainable_params())

c:\max\proga\vkmade\homeworks\vk_cv\src\architectures\effnet.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


12

In [6]:
dataset.sampling_type = "augmented"
history = train(
    model=model,
    dataset=dataset,
    lr=5e-3,
    batch_size=256,
    epoch_num=1,
    trainable_params=model.get_trainable_params(),
    tensorboard=True,
    iter_to_stop=100
)

  0%|          | 0/178 [00:00<?, ?it/s]

In [7]:
for param in model.model.features[7].parameters():
    param.requires_grad = True
    
for param in model.model.features[8].parameters():
    param.requires_grad = True


In [8]:
len(model.get_trainable_params())

80

In [14]:
model = load_model(
    model_name=os.environ["MODEL_NAME"]
)

In [16]:
dataset.sampling_type = "augmented"
history = train(
    model=model,
    dataset=dataset,
    lr=5e-7,
    batch_size=128,
    epoch_num=1,
    trainable_params=model.get_trainable_params(),
    tensorboard=True
)

  0%|          | 0/355 [00:00<?, ?it/s]

In [17]:
save_model(
    model=model,
    model_name=os.environ["MODEL_NAME"]
)

In [18]:
predict(os.environ["MODEL_NAME"])

Reading images:   0%|          | 0/45373 [00:00<?, ?it/s]

  0%|          | 0/19446 [00:00<?, ?it/s]